In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

#other computational libraries
import os
import sys
import numpy as np
import pandas as pd
import scipy.constants as co
import scipy.stats as ss
import itertools
import pickle
from scipy import signal

In [8]:
import ProjectFunctions as F
import ElasticCrossSections as el
import CaptureCrossSections as cap
import PeriodicTable as pt

In [2]:
# load data if it already exists

filespec = 'pnr_improved1'

NR = 300 # number of points in recoil energy grid to save
NN = 400 # number of points in neutron energy grid to save

currents_file = f'{filespec}_currents_dat.txt'
pRR_file = f'{filespec}_{NR}_pRR.txt'
pNR_file = f'{filespec}_{NR}_{NN}_pNR.txt'
ptot_file = f'{filespec}_{NR}_ptot.txt'
output_dir = 'papers/data/'

use_saved = True

ptot_loaded = False

print(currents_file)

if use_saved:
    
    if ptot_file in os.listdir(output_dir):
        with open(output_dir + ptot_file, 'rb') as f:
            ptot = pickle.load(f)
        ptot_loaded = True
    
if use_saved and currents_file in os.listdir(output_dir):
    with open(output_dir + currents_file, 'rb') as file:
        I, Fs = pickle.load(file)  
    
#I1 = F.interpolate_yvals_with_overlap(Ens, E_neut, spec['spec'].values)#np.exp(-Ens*10/Ens.max()) # incident flux   
#totalflux = F.trap(E_neut, spec['spec'].values)


pnr_improved1_currents_dat.txt


In [3]:
print(Fs)

[array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       9.12338598e-44, 2.50824966e-44, 0.00000000e+00]), array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       2.17832994e-46, 3.27996900e-47, 0.00000000e+00]), array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       4.10183503e-49, 3.17400736e-50, 0.00000000e+00]), array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       6.26271091e-52, 2.36700629e-53, 0.00000000e+00]), array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       7.93321987e-55, 1.39882870e-56, 0.00000000e+00]), array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       8.49210470e-58, 6.67899208e-60, 0.00000000e+00])]


In [4]:
print(ptot)

[[8.09128029e-05 8.09126675e-05 8.09125242e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
